In [13]:
#import thư viện
import h5py
import matplotlib.pyplot as plt
import numpy as np
import math
import seaborn as sns
import cv2

In [8]:
def grayscale(img):
    r, g, b = img[:, :, 0], img[:, :, 1], img[:, :, 2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return gray

In [9]:
#normalize
import math
import numpy as np
from copy import deepcopy

def getMean(img):
    '''Hàm tính trung vị ảnh đã gray scale'''
    mean = 0
    w, h = img.shape
    for i in range(w):
        for j in range(h):
            mean += img[i, j]
    return mean/(w*h)

def getVariance(img, _mean):
    '''Hàm tính phương sai ảnh đã gray scale dựa vào tham số là trung vị'''
    variance = 0
    w, h = img.shape
    for i in range(w):
        for j in range(h):
            variance += (img[i,j] - _mean)**2
    return variance/(w*h)

def normalize(img, nMean, nVar):
    '''Hàm để chuẩn hóa img đã gray scale từ trung vị và phương sai cũ 
    thành trung vị mới là nMean và phương sai mới là nVar'''
    w, h = img.shape
    _mean = getMean(img)
    var = getVariance(img, _mean)
    for i in range(w):
        for j in range(h):
            diff = math.fabs(img[i, j] - _mean)*math.sqrt(nVar/var)
            if img[i,j] > _mean:
                img[i, j] = nMean + diff
            else: img[i, j] = nMean - diff
    return img

In [10]:
def sobelY(image, i, j):
    return image[i + 2, j] + 2 * image[i + 2, j + 1] + image[i + 2, j + 2] - image[i, j] - 2 * image[i, j + 1] - image[i, j + 2]


def sobelX(image, i, j):
    return image[i, j + 2] + 2 * image[i + 1, j + 2] + image[i + 2, j + 2] - image[i, j] - 2 * image[i + 1, j] - image[i + 2, j]


def orientation_estimation(img, wS=4):
    width, height = img.shape
    #ft_img = np.zeros((height, width))
    direct = np.zeros((height, width))
    #g_kernel = np.zeros((height, width))
   
    for i in range(wS + 1, width - wS - 1):
        for j in range(wS + 1, height - wS - 1):
            gXX = gXY = gYY = 0
            for k in range(i - wS, i + wS - 1):
                for l in range(j - wS, j + wS - 1):
                    gX = sobelX(img, k, l)
                    gY = sobelY(img, k, l)
                    gXX += gX*gX
                    gXY += gX*gY
                    gYY += gY*gY
            direct[i, j] = 0.5*math.atan2(2*gXY,gXX-gYY)
    
            #g_kernel[i, j] = cv2.getGaborKernel((4, 4), 8.0, d, 10.0, 0.5, 0, ktype=cv2.CV_32F)
            #ft_img[i, j] = cv2.filter2D(img[wS+i, wS+j], cv2.CV_8UC3, g_kernel)
    return direct

def gabor_filter(img, wS=4):
    width, height = img.shape
    #ft_img = np.zeros((height, width))
    g_kernel = []
    direct = orientation_estimation(img, wS=4)
    
    for i in range(5, 250, 4):
        for j in range(5, 250, 4):
            g_kernel.append(cv2.getGaborKernel((4, 4), 8.0, direct[i][j], 10.0, 0.5, 0, ktype=cv2.CV_32F))
            #g_kernel[i, j] = cv2.getGaborKernel((4, 4), 8.0, d, 10.0, 0.5, 0, ktype=cv2.CV_32F)
            #ft_img[i, j] = cv2.filter2D(img[wS+i, wS+j], cv2.CV_8UC3, g_kernel)
    return g_kernel

In [14]:
import cv2
a = cv2.imread('hi.bmp')
gray = grayscale(a)
#gray1=  cv2.cvtColor(a, cv2.COLOR_BGR2GRAY)
#status= cv2.imwrite('123.bmp',gray)
#status1 = cv2.imwrite('124.bmp', gray1)

normal = normalize(gray, 50, 300)
print(normal.shape)

a[:, :, 0] = a[:, :, 1] = a[:, :, 2] = normal
#direct = orientation_estimation(normal)
#k_kernel = gabor_filter(normal)
#print(k_kernel)
#filtered_img = cv2.filter2D(a, cv2.CV_8UC3, g_kernel)
cv2.imshow('1',a)
#cv2.imshow('2', ft_img)
#cv2.imshow('3', gray1)
#cv2.waitKey()


mean =  125.09843146238114
var =  11915.171553172693
(103, 96)
